In [4]:
import pandas as pd
import psycopg2 as pg2

database = 'arch-snap_facebook-scale-1'
conn = pg2.connect("host='localhost'  port=5422 password='yuchao' dbname=%s"%database)

In [5]:
sql = "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';"
pd.read_sql(sql, conn)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,circles,duke,None,False,False,False,False
1,public,edges,duke,None,False,False,False,False
2,public,source_348_circle_circle3,duke,None,False,False,False,False
3,public,source_107_circle_circle5,duke,None,False,False,False,False
4,public,source_1912_circle_circle27,duke,None,False,False,False,False
...,...,...,...,...,...,...,...,...
319,public,botjoin_q_4path_b27cc5_node_2,duke,None,False,False,False,False
320,public,topjoin_q_4path_b27cc5_node_2,duke,None,False,False,False,False
321,public,topjoin_q_4path_b27cc5_node_3,duke,None,False,False,False,False
322,public,topjoin_q_4path_b27cc5_node_4,duke,None,False,False,False,False


In [9]:
sql = "SELECT count(*) FROM edges where source = '348';"
pd.read_sql(sql, conn)

,count
0,6384


In [10]:
sql = "SELECT count(*) FROM circles where source = '348';"
pd.read_sql(sql, conn)

,count
0,567


In [11]:
sql = "SELECT count(*) FROM edges where source = '348' group by fnode;"
pd.read_sql(sql, conn)

,count
0,47
1,9
2,4
3,60
4,50
...,...
219,71
220,42
221,57
222,11


In [7]:
sql = '''
SELECT
   TABLE_NAME, COLUMN_NAME
FROM
   information_schema.COLUMNS
   JOIN
       (SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema') AS tables
    ON tables.tablename = table_name
'''
pd.read_sql(sql, conn)

,table_name,column_name
0,circles,source
1,circles,circle
2,circles,node
3,edges,source
4,edges,fnode
...,...,...
720,relnsens_source_348_circle_9939a6,fnode
721,relnsens_source_348_circle_9939a6,tsens
722,topjoin_q_4star_568681_node_1,b
723,topjoin_q_4star_568681_node_1,a


In [30]:
sql = "SELECT * FROM circles;"
pd.read_sql(sql, conn)

,source,circle,node
0,0,0,71
1,0,0,215
2,0,0,54
3,0,0,61
4,0,0,298
...,...,...,...
4228,414,6,679
4229,414,6,638
4230,414,6,422
4231,414,6,663


In [95]:
sql = "SELECT * FROM edges;"
pd.read_sql(sql, conn)

,source,fnode,tnode
0,0,236,186
1,0,122,285
2,0,24,346
3,0,271,304
4,0,176,9
...,...,...,...
170169,414,594,563
170170,414,653,597
170171,414,584,679
170172,414,600,628


In [94]:
sql = "SELECT count(*) FROM circles group by source, circle;"
pd.read_sql(sql, conn)

,count
0,21
1,22
2,1
3,3
4,1
...,...
188,4
189,1
190,4
191,4


In [32]:
sql = "SELECT * FROM combined;"
pd.read_sql(sql, conn)

,fnode,tnode
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
88229,4026,4030
88230,4027,4031
88231,4027,4032
88232,4027,4038


In [23]:
# Count how many nodes appearing in circles
# Document: Nodes 4039
sql = '''
select count(distinct node) from circles;
'''
pd.read_sql(sql, conn)

,count
0,2884


In [26]:
# Count how many nodes appearing in edges
# Document: Nodes 4039
sql = '''
select count(distinct node) from (select fnode as node from edges UNION select tnode as node from edges) AS nodes;
'''
pd.read_sql(sql, conn)

,count
0,3959


In [40]:
# Count how many nodes appearing in combined
# Document: Nodes 4039
sql = '''
select count(distinct node) from (select fnode as node from combined UNION select tnode as node from combined) AS nodes;
'''
pd.read_sql(sql, conn)

,count
0,4039


In [36]:
# Count the unique edges
# Document: Edges 88234
sql = "SELECT COUNT(*) FROM (SELECT 1 FROM edges GROUP BY fnode, tnode) as uni;"
pd.read_sql(sql, conn)

,count
0,168486


In [12]:
# Count the union of edges and reversed edges
# Document: Edges 88234
sql = '''
WITH undirected_edges AS (SELECT * from edges UNION SELECT source, tnode as fnode, fnode as tnode from edges)
select count(*) from undirected_edges;
'''
pd.read_sql(sql, conn)

,count
0,170174


In [15]:
# Count the union of edges and reversed edges from combined
# Document: Edges 88234
sql = '''
WITH undirected_edges AS (SELECT * from combined UNION SELECT tnode as fnode, fnode as tnode from combined) 
select count(*) from undirected_edges;
'''
pd.read_sql(sql, conn)

,count
0,176468


In [16]:
# Count how many edges are the same in reverse
sql = '''
select count(*) from edges as e1 join edges as e2 on e1.fnode = e2.tnode AND e1.tnode = e2.fnode AND e1.source = e2.source;
'''
pd.read_sql(sql, conn)

,count
0,170174


In [17]:
# Count how many edges are the same in reverse without source restriction
sql = '''
select count(*) from edges as e1 join edges as e2 on e1.fnode = e2.tnode AND e1.tnode = e2.fnode;
'''
pd.read_sql(sql, conn)

,count
0,173634


In [5]:
# Count how many edges from combined are the same in reverse
sql = '''
select count(*) from combined as e1 join combined as e2 on e1.fnode = e2.tnode AND e1.tnode = e2.fnode;
'''
pd.read_sql(sql, conn)

,count
0,0


In [38]:
# Count the unique edges from combined
# Document: Edges 88234
sql = "SELECT COUNT(*) FROM (SELECT 1 FROM combined GROUP BY fnode, tnode) as uni;"
pd.read_sql(sql, conn)

,count
0,88234


In [21]:
# Count triangles from the same source
# Document: Number of triangles 1612010
sql = '''
select count(*) from edges as e1, edges as e2, edges as e3 WHERE e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode AND e1.source = e2.source AND e2.source = e3.source;
'''
pd.read_sql(sql, conn)

,count
0,9195840


In [22]:
# Count triangles
# Document: Number of triangles 1612010
sql = '''
select count(*) from edges as e1, edges as e2, edges as e3 WHERE e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode;
'''
pd.read_sql(sql, conn)

,count
0,9491958


In [16]:
# Count triangles
# Document: Number of triangles 1612010
sql = '''
WITH undirected_combined as (SELECT * from combined UNION select tnode as fnode, fnode as tnode from combined)
select count(*)/6 from undirected_combined as e1, undirected_combined as e2, undirected_combined as e3 WHERE e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode;
'''
pd.read_sql(sql, conn)

,?column?
0,1612010


In [7]:
# Test Union
sql = '''
select * from combined limit 1;
'''
pd.read_sql(sql, conn)

,fnode,tnode
0,0,1


In [11]:
# Test Union
sql = '''
(select tnode as fnode, fnode as tnode from combined limit 1)
UNION
(select * from combined limit 1);
'''
pd.read_sql(sql, conn)

,fnode,tnode
0,0,1
1,1,0


In [ ]:
# Count how many triangles such that all individuals share the common friend, and each of edge from different circle
# Output:
# source, circle 1, circle 2, circle 3, triangles
sql = '''
WITH full_info AS (
    SELECT 
        edges.source, fnode, c1.circle as fcircle, tnode, c2.circle as tcircle 
    FROM
        edges, circles c1, circles c2 where edges.source = c1.source AND edges.source = c2.source AND edges.fnode = c1.node AND edges.tnode = c2.node
)
select 
    count(*) 
from 
    full_info e1, full_info e2, full_info e3
WHERE
    e1.fcircle = e1.tcircle
    AND e2.fcircle = e2.tcircle
    AND e3.fcircle = e3.tcircle
    AND e1.fcircle <> e2.fcircle
    AND e2.fcircle <> e3.fcircle
    AND e1.fcircle <> e3.fcircle
    AND e1.source = e2.source
    AND e2.source = e3.source
    AND e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode
    ;
'''
pd.read_sql(sql, conn)

In [5]:
# Count how many triangles such that all individuals share the common friend, and each of edge from different circle
# Output:
# source, circle 1, circle 2, circle 3, triangles
sql = '''
WITH full_info AS (
    SELECT 
        edges.source, fnode, c1.circle as fcircle, tnode, c2.circle as tcircle 
    FROM
        edges, circles c1, circles c2 
    where edges.source = c1.source AND edges.source = c2.source AND edges.fnode = c1.node AND edges.tnode = c2.node
        AND c1.source = c2.source       
)
select 
    count(*) 
from 
    full_info e1, full_info e2, full_info e3
WHERE
    True
    AND e1.fcircle < e2.fcircle
    AND e2.fcircle < e3.fcircle
    AND e1.source = e2.source
    AND e2.source = e3.source
    AND e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode
    ;
'''
pd.read_sql(sql, conn)

,count
0,63679680


In [110]:
# Count how many triangles such that all individuals share the common friend, and each of edge from different circle
# Output:
# source, circle 1, circle 2, circle 3, triangles
sql = '''
WITH full_info AS (
    SELECT 
        edges.source, fnode, c1.circle as fcircle, tnode, c2.circle as tcircle 
    FROM
        edges, circles c1, circles c2 
    where edges.source = c1.source AND edges.source = c2.source AND edges.fnode = c1.node AND edges.tnode = c2.node
        AND c1.circle = c2.circle       
)
select 
    e1.source as source, e1.fcircle as circle1, e2.fcircle as circle2, e3.fcircle as circle3, count(*) as cnt 
from 
    full_info e1, full_info e2, full_info e3
WHERE
    True
    AND e1.fcircle < e2.fcircle
    AND e2.fcircle < e3.fcircle
    AND e1.source = e2.source
    AND e2.source = e3.source
    AND e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode
GROUP BY
    e1.source, e1.fcircle, e2.fcircle, e3.fcircle
;
'''
triangles = pd.read_sql(sql, conn)

In [111]:
triangles.groupby(['source'])['cnt'].agg('sum')

source
1912     4236
3437      134
348     63483
414        24
686     38563
698        24
Name: cnt, dtype: int64

In [112]:
triangles.sort_values(['cnt'], ascending=False)

,source,circle1,circle2,circle3,cnt
123,348,circle1,circle11,circle13,20751
124,348,circle1,circle11,circle2,10366
129,348,circle1,circle11,circle8,9374
104,348,circle0,circle1,circle11,8610
206,686,circle0,circle13,circle4,6235
...,...,...,...,...,...
39,1912,circle10,circle40,circle43,1
238,686,circle1,circle13,circle5,1
89,3437,circle0,circle14,circle3,1
33,1912,circle10,circle35,circle43,1


In [31]:
with pd.option_context("display.max_rows", 1000):
    display(triangles.query('cnt > 1000').sort_values(['cnt'], ascending=False))

,source,circle1,circle2,circle3,cnt
44,348,1,11,13,20751
20,348,1,2,11,10366
38,348,1,8,11,9374
4,348,0,1,11,8610
102,686,0,4,13,6235
99,686,0,4,9,4744
101,686,0,4,12,3800
189,686,4,9,13,3719
40,348,1,8,13,2270
120,686,0,9,13,1844


In [108]:
# Count how many triangles such that all individuals share the common friend, and each of edge from different circle
# Output:
# source, circle 1, circle 2, circle 3, triangles
sql = '''SELECT e1.source as ego1, e2.source ego2, e3.source ego3, count(*) as triangles FROM edges e1, edges e2, edges e3
    WHERE e1.source < e2.source AND e2.source < e3.source
    AND e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode
GROUP BY
    e1.source, e2.source, e3.source
;
'''
triangles_ego = pd.read_sql(sql, conn)

In [109]:
triangles_ego.sort_values(['triangles'], ascending=False)

,ego1,ego2,ego3,triangles
3,107,348,414,2899
4,1912,348,414,32
1,107,1912,348,6
2,107,1912,414,6
0,0,107,1684,3


In [113]:
def fetch_table(source, circle, rename):
    sql = '''
    WITH 
        edges AS (SELECT fnode, tnode FROM edges WHERE source = '{source}'),
        circles AS (SELECT node FROM circles WHERE source = '{source}' AND circle = '{circle}')
    SELECT fnode, tnode FROM edges WHERE fnode in (select * from circles) AND tnode in (select * from circles);
    '''.format(source=source, circle=circle)
    sql = ''
    return sql

def add_query(query):
    

source_limit = 10
sql = 'select source from circles group by source limit %d;'%source_limit
sources = pd.read_sql(sql, conn)['source'].tolist()

queries_limit = 20
queries = set()
tables = set()
for i in range(len(sources)):
    source = sources[i]
    sql = 'select circle from circles where source={source} group by circle;'.format(source=source)
    circles = pd.read_sql(sql, conn)['source'].tolist()
    if len(circles) < 4:
        continue
    target_circles = np.random.choice(circles, 4)
    for circle in target_circles:
        tables.add(circle)
    queries.add(target_circles)
    if len(queries) >= queries_limit:
        break

for query in queries:
    add_query(query)
for table in tables:
    add_table(table)
    

fetch_table('348', 'circle1', 'any')

,fnode,tnode
0,436,428
1,450,531
2,538,404
3,565,412
4,471,461
...,...,...
6241,397,389
6242,514,503
6243,527,412
6244,428,479


In [2]:
import sys
import pandas as pd

#!/usr/bin/python3

%load_ext autoreload
%autoreload 2

from run import run_TSens, run_Elastic, run_LTSens, run_Query, run_PrivateSQL, run_TSensDP
from algo import gen_report_title, gen_query_report_title
from objects import Relation
from dp import DP_TSens, DP_PrivateSQL
import dp
from import_schema import read_schema_from_file
from import_hypertree import read_hypertree_from_file

scale = '1'

def iterate_queries(plat, qtype):
    import os
    query_folder = './queries/snap_%s/'%plat
    queries = []
    for filename in os.listdir(query_folder):
        if filename[0] != 'q':
            continue
        qname = filename.strip().split('.')[0]
        qfullname = open(query_folder + filename, 'r').readline().strip().split(' ')[1]
        q = (qname, qfullname)
        if qtype == 'all':
            queries.append(q)
        else:
            qmark = qname.split('_')[1]
            if qtype == qmark:
                queries.append(q)
    return queries

limit_dict = {
    '4star': 30,
    'triangle': 100,
    '4cycle': 1000,
    '4path': 100000
}

def run_DP(plat, qtype = 'all', reps=1):
    old_stdout = sys.stdout
    #sys.stdout = open('/tmp/report', 'w')
    dp.gen_report_title()
    for qname, qfullname in iterate_queries(plat, qtype):
        arch = 'snap_%s'%plat
        limit = limit_dict[qname.split('_')[1]]
        eps = 1.0
        exclusion = []
        # Be careful ... it only works when each table is about source and circle ...
        source = qfullname.split('_')[2]
        circle = qfullname.split('_')[4]
        reln = Relation('any', 'source_{source}_circle_{circle}'.format(source=source, circle=circle), {})
        run_PrivateSQL(arch, scale, qname, reln, None, eps, report='file', reps=reps)
        break
    #sys.stdout = old_stdout
    #report = pd.read_csv('/tmp/report', sep=r' \| ', engine='python')
    #return report
    
run_DP('facebook')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
algo | arch | scale | query | reln | limit | reps | eps | pre_eps | run_eps | nosy_ans | gsens | bias_ans | true_ans | nosy_ans_list | bias_ans_list | gsens_list | avg_time | time_list
source_686_circle_circle12 []
{}
[Base Sens] source_686_circle_circle0 0
[Base Sens] source_686_circle_circle12 1
[Base Sens] source_686_circle_circle4 0
[Base Sens] tri_source_686_circle_circle13 0
[ESens] 996480
[Bias Ans] 1328
[Nosy Ans] 1484582.9782244998
[True Ans] 1328
PrivateSQL | snap_facebook | 1 | q_4star_9880d9 | source_686_circle_circle12 | None | 1 | 1.0 | 0.5 | 0.5 | 1484582.9782244998 | 996480.0 | 1328.0 | 1328 | [1484582.9782244998] | [1328] | [996480] | 0.16909503936767578 | [0.16909503936767578]


/home/BigDATA/TupleSens/dp.py:13: RuntimeWarning: divide by zero encountered in true_divide
  DIV = Infix(lambda x,y: np.divide(x,y))
